# Data Preparation

In [28]:
import psycopg2
import pandas as pd
import numpy as np
#import dbpass
pd.set_option('max_colwidth',0)

### Connecting to the Database

In [29]:
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME, user="postgres")
#conn = psycopg2.connect(dbname=DBNAME, user="postgres", password=dbpass.postgrepass())

### Query

We pull the data for the PUMAs (separate regions studied, defined by population and the edges of census tracts) that we are interested in.  We need employment information, education information, and ages.  We pull the data for everyone, not just opportunity youth so we can compare their prevalence to total population.

##### Choice of Regions

We decided to use 11610 - 11615 because on the PUMA_names look up table, they all say they are King Country South somewhere.  We also included 11604 and 11605 because those are clearly south of the City of Seattle.  As a long time Seattle resident, I know that the regions south of downtown tend to have lower property values, lower average incomes, and a greater representation of people of color.  To get a good picture of opportunity youth from this area, we needed to include those regions.  Also, they are included in the 'Opportunity Youth in the Road Map Project Region' report.

##### Sample Weights

Finally, we pull the sample weights, since only a fraction of the population was polled for this dataset.

In [30]:
query = """
SELECT cow, esr, schl education_attained, fschp enrollment_status, agep age, pwgtp sample_weight
FROM pums_2017
where puma between '11610' and '11615'
or puma between '11604' and '11605'
"""
df = pd.read_sql(query, conn)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45511 entries, 0 to 45510
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   cow                 28651 non-null  object 
 1   esr                 37100 non-null  object 
 2   education_attained  43980 non-null  object 
 3   enrollment_status   45511 non-null  object 
 4   age                 45511 non-null  float64
 5   sample_weight       45511 non-null  float64
dtypes: float64(2), object(4)
memory usage: 2.1+ MB


#### Data Cleaning

We see that cow (Class of Worker) and esr (employment recode, meant to differentiate between civilian and military workers) have missing values, as well as education_attained.

##### Employment

For our employment data we want as complete a dataset as possible, so we combine what we have in cow and esr to make a new column with a binary value.  Either the sample is employed or not; we don't care what kind of employment they have.

We set the samples marked as unemployed in either column to 0.

Then we impute the remaining missing values to be unemployed.  We made this choice for 2 reasons:

1. We assume folks are more likely to report that they are employed than if they are not.
2. We want to avoid under-countring our opportunity youths.

##### Education Attained

For this column it's less clear what missing values might represent.  There are many fewer missing values here, and less chance of severely undercounting our opportunity youth.  Here we choose to take
the mode of the column, representing the most common values.

In [32]:
# Create a new column and fill it with 1s, representing 'employed'.  This initiates our employed column.
df['employed'] = 1

# First we find the unemployed amoung the cow and esr features.  We set those samples to 0, representing 'unemployed'.
df.loc[(df.cow == '3') | (df.esr == '9'),'employed'] = 0

# Once we've pieced together as much data on the unemployed as we can from cow and esr, 
# we assume the missing values are unemployed.
df.loc[(df.cow == np.nan) & (df.esr == np.nan), 'employed'] = 0

# We then drop the cow and esr columns, because have extracted the information we need from them
# Into the employed column.
df.drop(columns = ['cow','esr'], inplace = True)


df['education_attained'].fillna(df['education_attained'].mode()[0],inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45511 entries, 0 to 45510
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   education_attained  45511 non-null  object 
 1   enrollment_status   45511 non-null  object 
 2   age                 45511 non-null  float64
 3   sample_weight       45511 non-null  float64
 4   employed            45511 non-null  int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ MB


We've filled our missing values.

In [33]:
df.describe()

,age,sample_weight,employed
count,45511.000000,45511.000000,45511.000000
mean,39.848103,22.602382,0.958054
std,22.858447,15.931955,0.200468
min,0.000000,1.000000,0.000000
25%,21.000000,13.000000,1.000000
50%,40.000000,18.000000,1.000000
75%,58.000000,25.000000,1.000000
max,94.000000,267.000000,1.000000


In [34]:
df.groupby('age').count()

,education_attained,enrollment_status,sample_weight,employed
age,,,,
0.0,497,497,497,497
1.0,524,524,524,524
2.0,510,510,510,510
3.0,542,542,542,542
4.0,558,558,558,558
...,...,...,...,...
87.0,127,127,127,127
88.0,122,122,122,122
89.0,84,84,84,84


#### Age == 0

It's tempting to think age == 0 might be a placeholder, but when I look at the distribution, it seems in line with other young children.  I'm going to assume it means an infant and leave them.

In [35]:
df['education_attained'].unique()

array(['05', '06', '04', '11', '22', '19', '16', '20', '21', '09', '18',
       '23', '14', '13', '07', '17', '08', '12', '01', '15', '02', '10',
       '03', '24'], dtype=object)

#### Education Attained

These are all valid values.

In [36]:
df.head()

,education_attained,enrollment_status,age,sample_weight,employed
0,05,0,40.0,90.0,1
1,06,0,11.0,78.0,1
2,04,0,9.0,60.0,1
3,11,0,48.0,109.0,1
4,11,0,48.0,108.0,1


In [37]:
conn.close()